In [1]:
import pandas as pd
import fnmatch

In [2]:
df = pd.read_csv("Resources/Parkingtickets_Dec2016.csv")
df.head()

,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Violation Time,Violation County,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Meter Number,Violation Post Code,Violation Description
0,GYG8911,NY,PAS,12/18/2016,24,SUBN,JEEP,1204P,Q,NaN,NaN,BK,2015,NaN,15 4,24-No Parking (exc auth veh)
1,8P82H,NY,OMT,12/21/2016,14,TAXI,NISSA,1015A,NY,ALL,ALL,YELLO,2014,-,NaN,NaN
2,GML6346,NY,PAS,12/19/2016,21,SDN,HONDA,1212P,K,1130A,0100A,GREY,2013,-,NaN,NaN
3,12203MG,NY,COM,12/1/2016,46,VAN,FRUEH,1137A,NY,ALL,ALL,BROWN,2015,-,NaN,NaN
4,HEG7444,NY,PAS,12/1/2016,36,4DSD,BMW,1027A,BK,NaN,NaN,BK,2016,NaN,NaN,PHTO SCHOOL ZN SPEED VIOLATION


# Data Cleanup

In [3]:
# Removes DK (dark) and LT (light) demarkations for colors and drops any non-string values
for index, row in df.iterrows():
    try:
        if row["Vehicle Color"].startswith("D"):
            if row["Vehicle Color"].startswith("DK"):
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("DK")
            else:
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("D")
            
        elif row["Vehicle Color"].startswith("L"):
            if row["Vehicle Color"].startswith("LT"):
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("LT")
            else:
                df.loc[index, "Vehicle Color"] = df.loc[index, "Vehicle Color"].strip("L")
        
    except AttributeError:
        df.drop(index, inplace=True)

In [4]:
# Defines standard color abbreviations as keys and irregular color abbreviations as values
colors = {
    "BK": ('BLA', 'BK'),
    "BL": 'BL',
    "BR": ('BR', 'BN'),
    "GY": ("GY", "GRA", "GREY", "GRY", "GEY", "S", "CH", "CR"), # combines grays and silvers
    "GN": ("GRE", "GN", "GRN"),
    "OR": "O",
    "RD": ("R", "BRG", "BUR", "M"), # combines reds, burgundies, and maroons
    "TN": ("T", "BE", "BG", "BI"), # combines tans and beiges
    "WH": "W",
    "YE": "Y"
}

# Loops through keys and changes irregular abbreviations to standard
for color in colors:
    df["Vehicle Color"].loc[df['Vehicle Color'].str.startswith(colors[color], na=False)==True] = color


C:\Users\Elliott\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
# Shows new color groupings
df['Vehicle Color'].value_counts()

BK       8423
GY       7767
WH       7753
BL       2478
RD       1611
BR        955
YE        526
GR        274
TN        187
OR        182
GN         68
G          54
GL         39
/          26
PR         22
GOLD       13
B          11
PURPL       8
UNKNO       6
NOCL        4
UNK         4
PLE         2
XXX         1
CM          1
AVEN        1
FREIG       1
CL          1
HONDA       1
P           1
NAVY        1
NOC         1
GLBR        1
Name: Vehicle Color, dtype: int64

# Plots

In [6]:
# Groups all colors outside top 10 as "Other"
